## Setup

In [1]:
# Make sure we're using the latest version
import Pkg
Pkg.add("BenchmarkTools")
Pkg.activate("../..")  # Activate the project environment
Pkg.instantiate()      # Install any missing dependencies
Pkg.status()          # Check if MyMlp is listed

# Now try importing
using BenchmarkTools
using LinearAlgebra

   Resolving package versions...
      Compat entries added for 
  No Changes to `~/Repos/AWiD/MyMlp/Project.toml`
  No Changes to `~/Repos/AWiD/MyMlp/Manifest.toml`


Project 

  Activating project at `~/Repos/AWiD/MyMlp`


MyMlp v0.1.0
Status `~/Repos/AWiD/MyMlp/Project.toml`
  [6e4b80f9] BenchmarkTools v1.6.0


## Comparison of different optimizations to Variable

In [1]:
import Pkg
Pkg.add("BenchmarkTools")
using BenchmarkTools

abstract type GraphNode end
abstract type Operator <: GraphNode end

# Original implementation
mutable struct VariableOriginal <: GraphNode
    output :: Any
    grad :: Any
    name :: String
    VariableOriginal(output; name="?") = new(output, nothing, name)
end

# Optimized implementation
mutable struct VariableOptimized{T<:Float64} <: GraphNode
    output :: T
    grad :: Union{Nothing, T}
    name :: String
    VariableOptimized(output::T; name="?") where T<:Float64 = new{T}(output, nothing, name)
end

# RefValue-based immutable implementation
struct VariableRef <: GraphNode
    output :: Base.RefValue{Float64}
    grad   :: Union{Nothing, Base.RefValue{Float64}}
    name   :: String
    function VariableRef(output::Float64; name="?")
        new(Base.RefValue(output), nothing, name)
    end
end

# Functions to benchmark: reading
function read_output(v::VariableOriginal)
    v.output + 1.0
end

function read_output(v::VariableOptimized)
    v.output + 1.0
end

function read_output(v::VariableRef)
    v.output[] + 1.0
end

# Functions to benchmark: writing
function write_output!(v::VariableOriginal)
    v.output = v.output + 1.0
end

function write_output!(v::VariableOptimized)
    v.output = v.output + 1.0
end

function write_output!(v::VariableRef)
    v.output[] = v.output[] + 1.0
end

# Create instances
v1 = VariableOriginal(1.0)
v2 = VariableOptimized(1.0)
v3 = VariableRef(1.0)

# Benchmark READ
println("### Benchmark: READ access ###")
println("Original:")
@btime read_output($v1)

println("Optimized:")
@btime read_output($v2)

println("RefValue:")
@btime read_output($v3)

# Benchmark WRITE
println("\n### Benchmark: WRITE mutation ###")
println("Original:")
@btime write_output!($v1)

println("Optimized:")
@btime write_output!($v2)

println("RefValue:")
@btime write_output!($v3)



   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


### Benchmark: READ access ###
Original:
  14.034 ns (1 allocation: 16 bytes)
Optimized:
  1.575 ns (0 allocations: 0 bytes)
RefValue:
  1.793 ns (0 allocations: 0 bytes)

### Benchmark: WRITE mutation ###
Original:
  14.417 ns (1 allocation: 16 bytes)
Optimized:
  2.013 ns (0 allocations: 0 bytes)
RefValue:
  2.013 ns (0 allocations: 0 bytes)


500503.0

## Next

In [10]:
import Base: *, +, clamp, log, exp
import LinearAlgebra: mul!
import Statistics: sum

abstract type GraphNode end
abstract type Operator <: GraphNode end

# Definition of basic structures for computational graph
struct Constant{T<:AbstractVecOrMat{Float64}} <: GraphNode
    output :: T
end

mutable struct Variable{T<:AbstractVecOrMat{Float64}} <: GraphNode
    output :: T
    gradient :: Union{Nothing, T}
    name :: String
    
    Variable(output::T; name="?") where {T<:AbstractVecOrMat{Float64}} = new{T}(output, nothing, name)
end

mutable struct ScalarOperator{F} <: Operator
    inputs :: Union{Nothing, Tuple{GraphNode, GraphNode}}
    output :: Union{Nothing, Float64}
    gradient :: Union{Nothing, Float64}
    name :: String
    ScalarOperator(fun, inputs...; name="?") = new{typeof(fun)}(inputs, nothing, nothing, name)
end

mutable struct BroadcastedOperator{F} <: Operator
    inputs :: Union{Nothing, Tuple{GraphNode, GraphNode}, Tuple{GraphNode}}
    output :: Union{Nothing, AbstractVecOrMat{Float64}}
    gradient :: Union{Nothing, Matrix{Float64}}
    name :: String
    BroadcastedOperator(fun, inputs...; name="?") = new{typeof(fun)}(inputs, nothing, nothing, name)
end


import Base: show, summary
show(io::IO, x::ScalarOperator{F}) where {F} = print(io, "op ", x.name, "(", F, ")");
show(io::IO, x::BroadcastedOperator{F}) where {F} = print(io, "op.", x.name, "(", F, ")");
show(io::IO, x::Constant) = print(io, "const ", x.output)
show(io::IO, x::Variable) = begin
    print(io, "var ", x.name);
    print(io, "\n ┣━ ^ "); summary(io, x.output)
    print(io, "\n ┗━ ∇ ");  summary(io, x.gradient)
end


function visit(node::GraphNode, visited, order)
    if node ∈ visited
    else
        push!(visited, node)
        push!(order, node)
    end
    return nothing
end

function visit(node::Operator, visited, order)
    if node ∈ visited
    else
        push!(visited, node)
        for input in node.inputs
            visit(input, visited, order)
        end
        push!(order, node)
    end
    return nothing
end

function topological_sort(head::GraphNode)
    visited = Set()
    order = Vector()
    visit(head, visited, order)
    return order
end


# x * y (aka matrix multiplication)
*(A::GraphNode, x::GraphNode) = BroadcastedOperator(mul!, A, x)
forward(::BroadcastedOperator{typeof(mul!)}, A, x) = return A * x
backward(::BroadcastedOperator{typeof(mul!)}, A, x, g) = tuple(g * x', A' * g)

# relu activation
relu(x::GraphNode) = BroadcastedOperator(relu, x)
forward(::BroadcastedOperator{typeof(relu)}, x) = return x .* (x .> 0.0)
backward(::BroadcastedOperator{typeof(relu)}, x, g) = tuple(g .* (x .> 0.0), nothing)

# add operation (for bias)
+(x::GraphNode, y::GraphNode) = BroadcastedOperator(+, x, y)
forward(::BroadcastedOperator{typeof(+)}, x, y) = return x .+ y
backward(::BroadcastedOperator{typeof(+)}, x, y, g) = begin
    grad_wrt_x = g
    # Gradient biasu: sumowanie gradientu g wzdłuż wymiaru batcha (dims=1)
    # i przekształcenie wyniku 1xN do wektora N-elementowego.
    grad_wrt_y = vec(sum(g, dims=2))
    return (grad_wrt_x, grad_wrt_y)
end

# sigmoid activation
σ(x::GraphNode) = BroadcastedOperator(σ, x)
forward(::BroadcastedOperator{typeof(σ)}, x) = return 1.0 ./ (1.0 .+ exp.(-x))
backward(node::BroadcastedOperator{typeof(σ)}, x, g) = begin
    y = node.output
    local_derivative = y .* (1.0 .- y)
    grad_wrt_x = g .* local_derivative
    return (grad_wrt_x, nothing)
end

# # binarycrossentropy
# binarycrossentropy(y::GraphNode, ŷ::GraphNode) = ScalarOperator(binarycrossentropy, y, ŷ)
# forward(::ScalarOperator{typeof(binarycrossentropy)}, ŷ, y) = begin
#     return -mean(y .* log.(ŷ) + (1.0 .- y) .* log.(1.0 .- ŷ))
# end
# backward(::ScalarOperator{typeof(binarycrossentropy)}, ŷ, y, g) = begin
#     J = (ŷ .- y) ./ (ŷ .* (1.0 .- ŷ))
#     return (J * g, nothing)
# end

function binary_cross_entropy_loss_impl(ŷ, y_true; epsilon=1e-10)
    ŷ_clamped = clamp.(ŷ, epsilon, 1.0 - epsilon)
    loss_elements = -y_true .* log.(ŷ_clamped) .- (1.0 .- y_true) .* log.(1.0 .- ŷ_clamped)
    return mean(loss_elements)
end

binarycrossentropy(ŷ::GraphNode, y::GraphNode) = ScalarOperator(binary_cross_entropy_loss_impl, ŷ, y)

forward(::ScalarOperator{typeof(binary_cross_entropy_loss_impl)}, ŷ_value, y_value) = begin
    loss_value = binary_cross_entropy_loss_impl(ŷ_value, y_value)
    return loss_value
end

backward(::ScalarOperator{typeof(binary_cross_entropy_loss_impl)}, ŷ_value, y_value, g) = begin
    epsilon = 1e-10
    ŷ_clamped_for_grad = clamp.(ŷ_value, epsilon, 1.0 - epsilon)
    local_grad_per_sample = (ŷ_clamped_for_grad .- y_value) ./ (ŷ_clamped_for_grad .* (1.0 .- ŷ_clamped_for_grad))
    batch_size = size(y_value, 2)
    grad_wrt_ŷ = local_grad_per_sample ./ batch_size
    return (grad_wrt_ŷ, nothing)
end


backward (generic function with 5 methods)

In [3]:
reset!(node::Constant) = nothing

reset!(node::Variable) = node.gradient = nothing
reset!(node::Operator) = node.gradient = nothing

compute!(node::Constant) = nothing
compute!(node::Variable) = nothing

compute!(node::Operator) =
    node.output = forward(node, [input.output for input in node.inputs]...)

function forward!(order::Vector)
    #   Iteruje przez każdy węzeł w order.
    for node in order
        compute!(node)
        reset!(node)
    end
    return last(order).output
end

forward! (generic function with 1 method)

In [4]:
update!(node::Constant, gradient) = nothing

update!(node::GraphNode, gradient) = if isnothing(node.gradient)
    node.gradient = gradient else node.gradient .+= gradient
end


function backward!(order::Vector; seed=1.0)
    result = last(order)   #   The output node

    if isa(result.output, Matrix{Float64})
        result.gradient = ones(Float64, size(result.output))
    else
        result.gradient = seed
        @assert length(result.output) == 1 "Gradient is defined only for scalar functions"
    end

    for node in reverse(order)   #   Iterate through nodes in reverse topological order.
        backward!(node)   #   Compute and propagate gradients backwards.
    end
    return nothing
end

function backward!(node::Constant) end
function backward!(node::Variable) end

function backward!(node::Operator)
    inputs = node.inputs

    gradients = backward(node, [input.output for input in inputs]..., node.gradient)

    for (input, gradient) in zip(inputs, gradients)
        update!(input, gradient)
    end
    return nothing
end

backward! (generic function with 4 methods)

## Optymalizator ADAM

In [ ]:
mutable struct AdamOptimizer
    α::Float64          # Learning rate
    β1::Float64         # Decay rate for first moment
    β2::Float64         # Decay rate for second moment
    ϵ::Float64          # Small constant for numerical stability
    t::Int              # Iteration counter

    # Stany m i v dla każdego parametru
    # IdDict pozwala na użycie obiektów jako kluczy (np. zmiennych parametrów)
    m_states::IdDict{Any, Any} # Mapa: parametr -> stan m (ta sama struktura co parametr)
    v_states::IdDict{Any, Any} # Mapa: parametr -> stan v (ta sama struktura co parametr)

    # Konstruktor optymalizatora
    function AdamOptimizer(α=0.001, β1=0.9, β2=0.999, ϵ=1e-8)
        new(α, β1, β2, ϵ, 0, IdDict(), IdDict())
    end
end



## Test wejścia do neuronu

In [6]:
x = Variable(reshape([1.0, 2.0, 3.0, 1.0, 2.0, 3.0], 3, 2), name="x")
x.output

3×2 Matrix{Float64}:
 1.0  1.0
 2.0  2.0
 3.0  3.0

In [7]:
w = Variable([1.0 2.0 3.0], name="w")
w.output

1×3 Matrix{Float64}:
 1.0  2.0  3.0

In [8]:
z = w * x
z.name = "z"

"z"

In [9]:
order = topological_sort(z)
println("Topological order:")
order


Topological order:


3-element Vector{Any}:
 var w
 ┣━ ^ 1×3 Matrix{Float64}
 ┗━ ∇ Nothing
 var x
 ┣━ ^ 3×2 Matrix{Float64}
 ┗━ ∇ Nothing
 op.z(typeof(mul!))

In [10]:
y = forward!(order)
z.output

UndefVarError: UndefVarError: `forward!` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [10]:
backward!(order)
w.gradient

1×3 Matrix{Float64}:
 2.0  4.0  6.0

## Test 2 szeregowych Neuronów - 1 warstwa + bias


In [5]:
x = Variable([1.0 1.0; 2.0 2.0; 3.0 3.0], name="x")
w = Variable([2.0 4.0 6.0; 3.0 5.0 7.0], name="w")
y = Constant(reshape([1.0, 1.0], 1, 2))
z = w * x
z.name = "z"
# c = Constant(1.0)
# d = z + c
# dense_layer_2 = σ(z)
# dense_layer_2.name = "σ(z)"
dense_layer_2 = relu(z)
dense_layer_2.name = "relu(z)"
loss = binarycrossentropy(dense_layer_2, y)
loss.name = "binarycrossentropy"
order = topological_sort(loss)
y = forward!(order)
backward!(order)

## Test 2. warstw neuronów 2-4

In [ ]:
#   Pierwsza warstwa
x = Variable(reshape([1.0, 2.0, 3.0], 3, 1), name="x")
w = Variable(reshape([2.0, 3.0, 4.0, 5.0, 6.0, 7.0], 2, 3), name="w1")
a = w * x
a.name = "a"
b = relu(a)
b.name = "b"

#   Druga warstwa
w2 = Variable(reshape([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0], 4,2), name="w2")
c = w2 * b
c.name = "c"
d = relu(c)
d.name = "d"
order = topological_sort(d)

7-element Vector{Any}:
 var w2
 ┣━ ^ 4×2 Matrix{Float64}
 ┗━ ∇ Nothing
 var w1
 ┣━ ^ 2×3 Matrix{Float64}
 ┗━ ∇ Nothing
 var x
 ┣━ ^ 3×1 Matrix{Float64}
 ┗━ ∇ Nothing
 op.a(typeof(mul!))
 op.b(typeof(relu))
 op.c(typeof(mul!))
 op.d(typeof(relu))

In [12]:
ŷ = forward!(order)
backward!(order)

## Test binary cross entropy

In [5]:
ŷ = Variable(reshape([0.8], 1, 1), name="ŷ")
y = Variable(reshape([1.0], 1, 1), name="y")
loss = binarycrossentropy(ŷ, y)
loss.name = "loss"
order = topological_sort(loss)
result = forward!(order)


0.2231435513142097

In [6]:
backward!(order)

##  Test tworzenia modelu dla batch = 2 relu-sigmoid-bce

In [5]:
x = Constant([1.0 1.0; 2.0 1.0; 3.0 1.0])
w1 = Variable([0.1 0.2 0.3; 0.4 0.5 0.6], name="w1")
z1_mul = w1 * x
z1_mul.name = "z1_mul"

b1 = Variable([0.1; 0.2], name="b1")
z1 = z1_mul + b1
z1.name = "z1"

a1 = relu(z1)
a1.name = "a1"
w2 = Variable([0.5 -0.5], name="w2")
z2_mul = w2 * a1
z2_mul.name = "z2_mul"

b2 = Variable([0.0], name="b2")
z2 = z2_mul + b2

ŷ = σ(z2)
ŷ.name = "ŷ"

y = Constant([1.0 0.0])

loss = binarycrossentropy(ŷ, y)
loss.name = "loss"

"loss"

In [6]:
order = topological_sort(loss)

13-element Vector{Any}:
 var w2
 ┣━ ^ 1×2 Matrix{Float64}
 ┗━ ∇ Nothing
 var w1
 ┣━ ^ 2×3 Matrix{Float64}
 ┗━ ∇ Nothing
 const [1.0 1.0; 2.0 1.0; 3.0 1.0]
 op.z1_mul(typeof(mul!))
 var b1
 ┣━ ^ 2-element Vector{Float64}
 ┗━ ∇ Nothing
 op.z1(typeof(+))
 op.a1(typeof(relu))
 op.z2_mul(typeof(mul!))
 var b2
 ┣━ ^ 1-element Vector{Float64}
 ┗━ ∇ Nothing
 op.?(typeof(+))
 op.ŷ(typeof(σ))
 const [1.0 0.0]
 op loss(typeof(binary_cross_entropy_loss_impl))

In [11]:
result = forward!(order)

0.8755166955155294

In [12]:
w2.output

1×2 Matrix{Float64}:
 0.5  -0.5

In [13]:
backward!(order)